In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init
from torch import optim
import numpy as np

import random
import os

seed = 10

random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

In [2]:
TEST_FILE = "data/test.txt"
TRAIN_FILE = "data/train.txt"
WHOLE_FILE = "data/whole.txt"
F_VOCAB_FILE = "data/vocab.f.txt"
Q_VOCAB_FILE = "data/vocab.q.txt"

In [3]:
class Options:
    def __init__(self):
        self.rnn_size = 50
        self.dropout = False
        self.init_weight = 0.08
        self.decay_rate = 0.985
        self.learning_rate = 0.01
        self.plot_every = 10
        self.print_every = 50
        self.grad_clip = 5
        self.dropout = 0.2
        self.dropoutrec = 0
        
opt = Options()

In [4]:
class LSTM(nn.Module):
    def __init__(self, opt):
        super(LSTM, self).__init__()
        self.opt = opt
        self.i2h = nn.Linear(opt.rnn_size, 4 * opt.rnn_size)
        self.h2h = nn.Linear(opt.rnn_size, 4 * opt.rnn_size)
        if opt.dropoutrec > 0:
            self.dropout = nn.Dropout(opt.droputrec)
            
    def forward(self, x, prev_c, prev_h):
        gates = self.i2h(x) + self.h2h(prev_h)
        ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)
        ingate = torch.sigmoid(ingate)
        forgetgate = torch.sigmoid(forgetgate)
        cellgate = torch.tanh(cellgate)
        outgate = torch.sigmoid(outgate)
        if self.opt.dropoutrec > 0:
            cellgate = self.dropout(cellgate)
        cy = (forgetgate * prev_c) + (ingate * cellgate)
        hy = outgate * torch.tanh(cy)  # n_b x hidden_dim
        return cy, hy


In [5]:
class Encoder(nn.Module):
    def __init__(self, opt, input_size):
        super(Encoder, self).__init__()
        self.opt = opt
        self.hidden_size = opt.rnn_size
        self.embedding = nn.Embedding(input_size, self.hidden_size)
        self.lstm = LSTM(self.opt)
        if opt.dropout > 0:
            self.dropout = nn.Dropout(opt.dropout)
        self.__initParameters()

    def __initParameters(self):
        for name, param in self.named_parameters():
            if param.requires_grad:
                init.uniform_(param, -opt.init_weight, opt.init_weight)
                
    def forward(self, input_src, prev_c, prev_h):
        src_emb = self.embedding(input_src) # batch_size x src_length x emb_size
        if self.opt.dropout > 0:
            src_emb = self.dropout(src_emb)
        prev_cy, prev_hy = self.lstm(src_emb, prev_c, prev_h)
        return prev_cy, prev_hy

In [6]:
class Decoder(nn.Module):
    def __init__(self, opt, output_size):
        super(Decoder, self).__init__()
        self.opt = opt
        self.hidden_size = opt.rnn_size

        self.embedding = nn.Embedding(output_size, self.hidden_size)
        self.lstm = LSTM(self.opt)
        self.linear = nn.Linear(self.hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        if opt.dropout > 0:
            self.dropout = nn.Dropout(opt.dropout)
        self.__initParameters()

    def __initParameters(self):
        for name, param in self.named_parameters():
            if param.requires_grad:
                init.uniform_(param, -opt.init_weight, opt.init_weight)
                
    def forward(self, input, prev_c, prev_h):
        output = self.embedding(input)
        if self.opt.dropout > 0:
            output = self.dropout(output)
        next_c, next_h = self.lstm(output, prev_c, prev_h)
        if self.opt.dropout > 0:
            next_h = self.dropout(next_h)
        h2y = self.linear(next_h)
        pred = self.softmax(h2y)
        return pred, next_c, next_h

In [7]:
def read_data(fh):
    for line in fh:
        sentence, lf = line.strip().split("\t")
        sentence = sentence.split()
        lf = lf.split()
        yield sentence, lf

def read_vocab(filename):
    t2i = {"<s>": 0, "</s>":1, "UNK": 2}
    with open(filename) as target:
        for line in target:
            token = line.strip().split()[0]
            if token not in t2i:
                t2i[token] = len(t2i)
    return t2i

def is_equal(gold, predictions):
    total_correct = 0.0
    if len(gold) == len(predictions):
        equal = True
        for g, p in zip(gold, predictions):
            if g != p:
                equal = False
        return equal
    return False

In [8]:
def preprare_data(file_name):
    shuffledData = None
    with open(TRAIN_FILE, 'r') as train:
        shuffledData = list(read_data(train))
        random.shuffle(shuffledData)
    sentence_index_tensors = []
    form_index_tensors = []
    for sentence in shuffledData:
        text_tensor = torch.zeros((1, len(sentence[0]) + 2), dtype=torch.long)
        text_tensor[0][0] = w2i["<s>"]
        for idx, word in enumerate(sentence[0]):
            word_index = w2i[word] if word in w2i else w2i["UNK"]
            text_tensor[0][idx+1] = word_index
        text_tensor[0][-1] = w2i["</s>"]
        sentence_index_tensors.append(text_tensor)
        form_tensor = torch.zeros((1, len(sentence[1]) + 2), dtype=torch.long)
        form_tensor[0][0] = lf2i["<s>"]
        for idx, form in enumerate(sentence[1]):
            form_index = lf2i[form] if form in lf2i else lf2i["UNK"]
            form_tensor[0][idx+1] = form_index
        form_tensor[0][-1] = lf2i["</s>"]
        form_index_tensors.append(form_tensor)
    return shuffledData, sentence_index_tensors, form_index_tensors

In [9]:
def train(opt, criterion, encoder_optimizer, decoder_optimizer, encoder, decoder, s1, f1):
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    c = torch.zeros((1, opt.rnn_size), dtype=torch.float, requires_grad=True)
    h = torch.zeros((1, opt.rnn_size), dtype=torch.float, requires_grad=True)
    for i in range(s1.size(1)):
        c, h = encoder(s1[:, i], c, h)

    #for dec_in in f1:
    loss = 0
    for i in range(f1.size(1)-1):
        pred, c, h = decoder(f1[:, i], c, h)
        loss += criterion(pred, f1[:, i+1])
    loss.backward()
    if opt.grad_clip != -1:
        torch.nn.utils.clip_grad_value_(encoder.parameters(),opt.grad_clip)
        torch.nn.utils.clip_grad_value_(decoder.parameters(),opt.grad_clip)
    encoder_optimizer.step()
    decoder_optimizer.step()
    return loss

In [10]:
def predict(opt, s1, lf2i, encoder, decoder):
    c = torch.zeros((1, opt.rnn_size), dtype=torch.float, requires_grad=True)
    h = torch.zeros((1, opt.rnn_size), dtype=torch.float, requires_grad=True)

    for i in range(s1.size(1)):
        c, h = encoder(s1[:, i], c, h)

    prev = torch.tensor([lf2i['<s>']], dtype=torch.long)
    predicted_form = []
    counter = 0
    while True:
        counter += 1
        pred, c, h = decoder(prev, c, h)
        form_id = pred.argmax().item()
        prev = torch.tensor([form_id], dtype=torch.long)
        if form_id == lf2i["</s>"] or counter >= 100:
            break
        predicted_form.append(form_id)
    return predicted_form

In [11]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker


def showPlot(points, fig_name, extra_info):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.title(extra_info) 
    plt.plot(points)
    plt.savefig("{}.png".format(fig_name))
    plt.close('all')

In [12]:
def train_and_test(epoch_num, directory):
    train_data, sentence_index_tensors_train, form_index_tensors_train = preprare_data(TRAIN_FILE)
    test_data, sentence_index_tensors_test, form_index_tensors_test = preprare_data(TEST_FILE)
    
    encoder = Encoder(opt, len(w2i))
    decoder = Decoder(opt, len(lf2i))

    optim_state = {"learningRate" : opt.learning_rate, "alpha" :  opt.decay_rate}
    encoder_optimizer = optim.RMSprop(encoder.parameters(),  lr=optim_state["learningRate"], alpha=optim_state["alpha"])
    decoder_optimizer = optim.RMSprop(decoder.parameters(),  lr=optim_state["learningRate"], alpha=optim_state["alpha"])
    criterion = nn.NLLLoss(ignore_index=0)

    losses = []
    max_acc = 0
    maxAccEpochId = 0
    accuracies = []
    for epoch in range(epoch_num):
        print("---Epoch {}---\n".format(epoch+1))
        print("Training...")
        encoder.train()
        decoder.train()
        plot_data = []
        for index, (sentence, form) in enumerate(zip(sentence_index_tensors_train, form_index_tensors_train)):
            loss = train(opt, criterion, encoder_optimizer, decoder_optimizer, encoder, decoder, sentence, form)
            if index != 0:
                if index % opt.plot_every == 0:     
                    plot_data.append(np.mean(losses[epoch*len(train_data)+index-opt.plot_every:]))
                if index % opt.print_every == 0:
                    print("Index {} Loss {}".format(index, np.mean(losses[epoch*len(train_data)+index-opt.print_every:])))
            losses.append(loss.item())

        print("Predicting..")
        encoder.eval()
        decoder.eval()
        correct = 0.0
        for index, (sentence, form) in enumerate(zip(sentence_index_tensors_test, form_index_tensors_test)):
            prediction = predict(opt, sentence, lf2i, encoder, decoder)
            prediction = [i2lf[p] for p in prediction]
            #print(test_data[index][1])
            #print(prediction)
            same = True
            for g, p in zip(test_data[index][1], prediction):
                if g != p:
                    same = False
            if same:
                correct += 1
                #print("Correct match ", prediction)
        accuracy = 100*(correct/len(test_data))
        accuracies.append(accuracy)
        if accuracy > max_acc:
            max_acc = accuracy
            maxAccEpochId = epoch
        print("Accuracy: ", accuracy)
        if not os.path.exists(directory):
            os.makedirs(directory)
        file_name = "{}/epoch.{}".format(directory, epoch)
        extra = "Mean Loss {0:.2f}".format(np.mean(losses))
        showPlot(plot_data, file_name, extra)
    file_name = "{}/{}".format(directory, "accuracies")
    extra = "Maximum Accuracy {0:.2f} at epoch {1}".format(np.max(accuracies), maxAccEpochId)
    showPlot(accuracies, file_name, extra)
    file_name = "{}/{}".format(directory, "all_losses")
    
    extra = "Mean Loss {0:.2f}".format(np.mean(losses))
    showPlot(losses, file_name, extra)

In [13]:
w2i = read_vocab(Q_VOCAB_FILE)
lf2i = read_vocab(F_VOCAB_FILE)
i2lf = {lf2i[i] : i for i in lf2i}

In [14]:
train_and_test(50, "out/WithGradientClippingAndDropout")

---Epoch 1---

Training...
Index 50 Loss 37.062820205688475
Index 100 Loss 22.35060528755188
Index 150 Loss 20.61336812019348
Index 200 Loss 17.10960817337036
Index 250 Loss 12.480404286384582
Index 300 Loss 15.243009309768677
Index 350 Loss 14.816191139221191
Index 400 Loss 17.51746823310852
Index 450 Loss 15.121167697906493
Index 500 Loss 16.21831046581268
Index 550 Loss 11.95041048526764
Predicting..
Accuracy:  0.0
---Epoch 2---

Training...
Index 50 Loss 15.567510018348694
Index 100 Loss 12.866727714538575
Index 150 Loss 14.111239376068115
Index 200 Loss 11.657528157234191
Index 250 Loss 9.001372170448303
Index 300 Loss 11.644767847061157
Index 350 Loss 11.676557731628417
Index 400 Loss 14.357117009162902
Index 450 Loss 12.578595099449158
Index 500 Loss 13.046538314819337
Index 550 Loss 10.925333523750306
Predicting..
Accuracy:  5.833333333333333
---Epoch 3---

Training...
Index 50 Loss 14.238794927597047
Index 100 Loss 11.32036850452423
Index 150 Loss 12.073881936073303
Index 200 

Index 50 Loss 9.779156978726387
Index 100 Loss 7.051431990861893
Index 150 Loss 7.553302385807037
Index 200 Loss 6.972201886773109
Index 250 Loss 5.146625252962113
Index 300 Loss 6.486664817929268
Index 350 Loss 7.60764814376831
Index 400 Loss 8.862344679832459
Index 450 Loss 7.185458706617355
Index 500 Loss 8.829319383352995
Index 550 Loss 6.588666895031929
Predicting..
Accuracy:  36.333333333333336
---Epoch 21---

Training...
Index 50 Loss 9.818297124505044
Index 100 Loss 6.708420813083649
Index 150 Loss 7.440986632108689
Index 200 Loss 6.207282903194428
Index 250 Loss 4.934462125301361
Index 300 Loss 7.186246987581253
Index 350 Loss 7.205182533264161
Index 400 Loss 8.93486205816269
Index 450 Loss 7.479788546562195
Index 500 Loss 8.777852116823196
Index 550 Loss 6.165952062606811
Predicting..
Accuracy:  34.0
---Epoch 22---

Training...
Index 50 Loss 10.097838764190675
Index 100 Loss 6.734761670827866
Index 150 Loss 8.053867215514183
Index 200 Loss 7.093232629299163
Index 250 Loss 4.6

Index 150 Loss 7.646412497758865
Index 200 Loss 6.181170696020127
Index 250 Loss 5.637913837432861
Index 300 Loss 7.367980968356132
Index 350 Loss 6.678414969742298
Index 400 Loss 9.37062117099762
Index 450 Loss 8.104602736234664
Index 500 Loss 8.438878471851348
Index 550 Loss 5.873487174510956
Predicting..
Accuracy:  31.333333333333336
---Epoch 40---

Training...
Index 50 Loss 9.947265850752592
Index 100 Loss 6.733190488815308
Index 150 Loss 7.3545608013868335
Index 200 Loss 6.596686439216137
Index 250 Loss 4.525652215480805
Index 300 Loss 6.944876497983932
Index 350 Loss 5.953017948269844
Index 400 Loss 8.294635705277324
Index 450 Loss 6.76594306319952
Index 500 Loss 8.587490468621255
Index 550 Loss 6.453695157170296
Predicting..
Accuracy:  30.333333333333336
---Epoch 41---

Training...
Index 50 Loss 9.624384880065918
Index 100 Loss 6.838113038763404
Index 150 Loss 7.467932903766632
Index 200 Loss 6.18791894055903
Index 250 Loss 5.174340655803681
Index 300 Loss 7.0265438920259475
Ind

In [15]:
"Maximum Accuracy {0:.2f} at epoch {1}".format(2.7888, 10)

'Maximum Accuracy 2.79 at epoch 10'